# Using a State-Of-The-Art (SOTA) GANs

In this exercise you will get to experiment with some modern and best-in-class GANs.

These GANs have been built with the [StyleGAN-3](https://nvlabs.github.io/stylegan3/) system by NVIDIA. Since here we will only use them for inference - no training necessary - we will not need a lot of code.

Let's start by importing all the modules we will need:

In [ ]:
import os
import pickle
import matplotlib.pyplot as plt
import torch

Now we can load one of the models. There are several GANs available for use directly from the authors. You can find the full list [here](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/research/models/stylegan3/files).

For this exercise, we are going to experiment with a StyleGAN trained on the [Animal Faces-HQ](https://paperswithcode.com/dataset/afhq) dataset. It can generate unconditionally images of animals.

Let's load the model. In this case, the authors decided to export their models in a special pickle file containing all the needed code:

> NOTE: in order to use it on your own, remember to download the [StyleGAN3](https://github.com/NVlabs/stylegan3) repo and put it in your PYTHONPATH or do this:

In [ ]:
# Add styleGAN3 repo to the paths where python
# searches for modules
import sys
sys.path.append("/home/student/stylegan3")

In [ ]:
with open('/home/student/.cache/dnnlib/downloads/20755e1ffb4380580e4954f8b0f9e630_stylegan3-r-afhqv2-512x512.pkl', 'rb') as f:
    stylegan_model = pickle.load(f)

Let's look at what's inside:

In [ ]:
stylegan_model.keys()

Can you guess what do the `G` and `D` stand for here?

> **YOUR ANSWER HERE**

What about `G_ema`?

> **YOUR ANSWER HERE**

We can ignore the rest of the keys. Ok, so let's get the Exponential Moving Average Generator then:

In [ ]:
G =  # YOUR CODE HERE: get the EMA model from the stylegan_model dictionary
G

We can see that the Generator network here is a lot more complex than what we used in our exercise from scratch. You can refere to the StyleGAN3 repo and paper for details.

We won't need it for generating, but just for fun, let's also get the Discriminator network and have a look at it:

In [ ]:
D = # YOUR CODE HERE: get the Discriminator
D

The Discriminator looks a lot more like a normal Convolutional Neural Network. Note however that the output of the discriminator is 1 number with a linear activation:

```
(out): FullyConnectedLayer(in_features=512, out_features=1, activation=linear)
```
instead of a Sygmoid activation as we used before. Why? Because this is actually a Critic network more than a Discriminator network, so the output is a score for the input image, and not a class probability as in the case of the DCGAN we have seen before.

Anyway, now we are ready to generate an image.

In [ ]:
z_dim = G.z_dim

# Generate a latent code with batch dimension of 1
# and latent dimension z_dim
# Remember: you can generate random numbers distributed as a 
# normal distribution using torch.randn
z = # YOUR CODE HERE

Now let's move everything to the GPU:

In [ ]:
z = z.to("cuda")
G = G.to("cuda")

Great, now call the Generator network:

> NOTE: StyleGAN3 can also be class-conditioned, i.e., conditioned to generate a specific class in the dataset. This model we are using is not, so our class tensor will just be None

In [ ]:
c = None
# Call the Generator passing in the latent.
img = # YOUR CODE HERE

The image generated by the network contains values distributed more or less in the range -1 to +1, but not exactly. We need to convert that to a RGB image and trucante the values that are outside of the range. We can do all of that like this:

In [ ]:
# From batch, bgr to batch, RGB
rgb_img = img.permute(0, 2, 3, 1)

# Convert to the RGB range then cut values outside of the range
rgb_img = (rgb_img * 127.5 + 128).clamp(0, 255)

# Now convert to unsigned integers and move to CPU, so we can display
rgb_img = rgb_img.to(torch.uint8).cpu()

In [ ]:
plt.imshow(rgb_img.squeeze(dim=0))
plt.axis("off")

Now generate a few more images:

In [ ]:
# YOUR CODE HERE


Look the details here! The fur and all the other details are reproduced almost flawlessly.